# Error modeling

The matching operation in this workbook takes a long time to run. You can run these cells later but please don't run them now since they will take hours to run.

We will run two experiments for error modeling.

1. Bypass experiment
2. Dextran pulse experiment

We will see how to setup a basic error model and how to carry the error from the bypass experiment over to the dextran pulse experiment.

## Standard imports

In [1]:
#from IPython.core.display import display, HTML
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

#Python path library support
from pathlib import Path

#python numeric library
import numpy

#python math library
import math

#scientific library for python
import scipy

#pandas is python library for data analysis
import pandas

#addict is a library that makes it easier to create nested dictionaries
from addict import Dict

#json is a standard text based format and it used in CADETMatch for the configuration file
import json

#python plotting library
import matplotlib.pyplot as plt

#cadet python interface
from cadet import Cadet

#jupyter widget support
from ipywidgets import interact
import ipywidgets as widgets

from CADETMatch.jupyter import Match

#This is just a support function to make running the examples a bit easier later
def deleteResultDir(path):
    #This is to try and make sure that only an actual result directory can be removed
    #And only if no other files have been added to the directory, just in case
    if not path.exists():
        return
    allowed = {'evo', 'grad', 'log', 'mcmc', 'meta', 'misc', 'progress', 
               'space', 'error.csv', 'result.h5', 'progress.csv'}
    allowed.add(match.csv)
    allowed.add(match_file.name)
    okayToRemove = False
    for child in path.iterdir():
        if child.name not in allowed:
            break
    else:
        okayToRemove = True
    if okayToRemove:
        import shutil
        shutil.rmtree(path)
        
def run_match(match_obj, match_file, true_values, delete):
    "create the matching object, run it and then look at output"
    match = Match(match_file.as_posix())

    resultDir = Path(match_obj.baseDir) / match_obj.resultsDir
    print("Your results are located at", resultDir.as_posix(), '\n\n')

    #If you want to delete the fitting process to start over remove the # from the next line and re-run this cell\
    if delete:
        deleteResultDir(resultDir)

    match.start_sim()

    match.plot_best()
    best ,score, best_score = match.get_best()
    for key,value in best_score.items():
        print("Method: %s" % key)
        for name, val, true_value in zip(match.cache.parameter_headers, value, true_values):
            print("%s = %.2e  true_value = %.2e error: %.2g%%" % (name, val, true_value, numpy.abs(val-true_value)/val*100))
        print('\n')
    return match

## Setup CADET path

In order to run examples we need to specify where CADET is.

On Windows just change the cadet_bin_path to the bin directory of CADET

On Linux you need to change the cadet_bin_path and also change cadet-cli.exe to cadet-cli on the following line.

In [3]:
#put in the path to the bin folder
cadet_bin_path = Path(r"C:\Users\kosh_000\cadet_build\CADET\VCPKG\bin")

cadet_path = cadet_bin_path / "cadet-cli.exe"

if cadet_path.exists():
    print("CADET was found. Continue to the next step.")
    Cadet.cadet_path = cadet_path.as_posix()
else:
    print('CADET could not be found. Please check the bin path')
    
print(cadet_path.as_posix())

CADET was found. Continue to the next step.
C:/Users/kosh_000/cadet_build/CADET/VCPKG/bin/cadet-cli.exe


## Create lesson folders and create lesson variables

This will create all the lesson folders we need and make it easier to refer to previous lessons

In [4]:
#set this to a directory on your computer where the cadet tutorial files can be saved
cadet_tutorial = Path('F:/temp/cadet_tutorial')
lesson_1 = cadet_tutorial / "lesson_1"
lesson_2 = cadet_tutorial / "lesson_2"
lesson_3 = cadet_tutorial / "lesson_3"
lesson_4 = cadet_tutorial / "lesson_4"
lesson_5 = cadet_tutorial / "lesson_5"
lesson_6 = cadet_tutorial / "lesson_6"
lesson_6.mkdir(exist_ok = True, parents=True)

# Create Bypass experiment

## Create configuration file

## Run match

## View results

# Create Dextran experiment

## Create configuration file

## Run match

## View results